# Python Technical Challenge

## _Searching an algorithm that sorts words from left to right line by line_

c: Dominic Catana <catanadominic@gmail.com>

In [83]:
# -----------------ONLY METHODS AND IMPORTS------------------------
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from operator import itemgetter

def extract_points(ocr_dataset):
    list_of_coords = []
    for it in ocr_dataset:
        for y in it['points']:
            list_of_coords.append(y)
    return list_of_coords

def get_ocr_data(path):
    f = open(path,)
    data = json.load(f)['ocr'];
    return data

def change_points_to_coords(ocr_dataset):
    l = []
    c = 0;
    x = -1;
    y = -1;
    min = 100000
    max = 0
    for c in ocr_dataset:
        x = c["points"][3][0]
        y = c["points"][3][1]
        w = c["points"][1][0]-c["points"][0][0]
        h = c["points"][2][1]-c["points"][1][1]
        c["points"]=[x,y,w,h]
    return ocr_dataset

def useful_data(coords):
    values = []
    original_list = []
    for i in coords:
        values.append(i['text'])
        values.append(i['points'][0])
        values.append(i['points'][2])   # take only the text and the opposite corners (left buttom, right top)
        original_list.append(values)
        values = []
    return original_list

def get_coords_sum(coords):
    values = []
    original_list = []
    for i in coords:
        values.append(i['text'])
        values.append(i['points'][0])
        values.append(i['points'][0][0]+i['points'][0][1])   # take text, sum of coordinates first point and top right corner's y
        values.append(i['points'][2][1])                   
        original_list.append(values)
        values = []
    return original_list 

def get_coords(sum_coords):
    values = []
    for i in sum_coords:
        values.append(i[1])  
    return values 
    

def sorting_algorithm(coords):
    initial_coords = useful_data(coords)
    sum_coords = get_coords_sum(coords)
    x_and_y = get_coords(sum_coords)
    return x_and_y
    



In [84]:
# --------------__main__--------------------

data = get_ocr_data("datasets/menu2.json")
#print(data)

print(sorting_algorithm(data))












[[133.87486267089844, 511.69140625], [110.84564971923828, 510.99517822265625], [89.71671295166016, 510.99517822265625], [191.95838928222656, 510.49652099609375], [179.46607767535104, 510.31321618621934], [221.04116821289062, 509.0448303222656], [247.25387573242188, 508.1828308105469], [270.950927734375, 508.54278564453125], [139.4509735107422, 490.1913146972656], [200.88742065429688, 489.5411682128906], [252.44467163085938, 488.0399169921875], [178.78329467773438, 463.2112731933594], [205.66726684570312, 463.3027038574219], [70.21308135986328, 454.75970458984375], [112.470947265625, 454.43463134765625], [93.29236602783203, 455.084716796875], [137.82566833496094, 454.43463134765625], [286.05328369140625, 450.2088623046875], [302.4812927246094, 450.3576965332031], [184.8527374267578, 417.1494140625], [196.14244079589844, 394.3113708496094], [97.84322357177734, 385.1967468261719], [71.51332092285156, 384.87164306640625], [122.22276306152344, 383.8965148925781], [199.58717346191406, 384.22